In [7]:
from flask import Flask, request, jsonify
import os
import serpapi
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
import numpy as np


In [8]:
app = Flask(__name__)



In [10]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('SERPAPI_KEY')
client = serpapi.Client(api_key=api_key)

In [ ]:
model = tf.keras.models.load_model("path_to_your_saved_model")


class_map = {
    0: "class_0",
    1: "class_1",
    # Add more class mappings as needed

}

In [11]:
def predictions(test_image_path):
    test_img = image.load_img(test_image_path, target_size=(224, 224))
    test_img_arr = image.img_to_array(test_img) / 255.0
    test_img_input = test_img_arr.reshape((1, test_img_arr.shape[0], test_img_arr.shape[1], test_img_arr.shape[2]))
    predicted_label = np.argmax(model.predict(test_img_input))
    predicted_product = class_map.get(predicted_label, "Unknown")
    return predicted_product

In [12]:
@app.route('/upload', methods=['POST'])
def upload_file():
    if 'image' not in request.files:
        return jsonify({"error": "No image provided"}), 400
    
    file = request.files['image']
    if file.filename == '':
        return jsonify({"error": "No image selected"}), 400

    # Save the uploaded image
    image_path = os.path.join("uploads", file.filename)
    file.save(image_path)

    # Make predictions
    predicted_product = predictions(image_path)

    # Scrape price information
    q_inf = predicted_product + " grocery store"
    results = client.search({
        'engine': 'google_shopping',
        'q': q_inf,
        'google_domain': 'google.ca',
        'gl': 'ca',
        'num': 20,
        'start': 1
    })

    prices = [product['price'] for product in results['shopping_results']]

    if prices:
        cheapest_price = min(prices)
    else:
        cheapest_price = "N/A"

    response_data = {
        "predicted_product": predicted_product,
        "cheapest_price": cheapest_price
    }

    return jsonify(response_data)

In [13]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\rashe\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
